<a href="https://colab.research.google.com/github/boramkim0514/KOIPA_AI/blob/main/07_%EB%AF%B8%EB%8B%88%EB%B0%B0%EC%B9%98_%ED%95%99%EC%8A%B5%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 배치란?
  * 데이터의 묶음
  * 묶음 대로 결과물이 계산 된다.
    * 100개 데이터를 한꺼번에 묶어서(배치를 만들어서) 입력을 했으면, 거기에 대한 결과물도 100개가 한꺼번에 나온다.
  * 배치를 적용한 Loss의 수식은?
  * `N`이면 `N`건에 대한 `CEE` 값을 구한 다음( 각각이라곤 했지만 한꺼번에 구해진다 )
  * 그 값들을 모두 더하고 `N`에 대한 평균을 구한다.

**배치를 적용한 CEE**
$$
CEE = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}\log{y_{nk}}
$$

# 미니배치란?
  * `MNIST`의 데이터의 개수는 60,000건
  * 신경망이 `MNIST`를 학습 하고, 거기에 대한 평가를 내릴 때 60,000건 모두에 대한 손실 함수의 합을 구해야 할까?
  * 데이

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [1]:
# # mnist 데이터세트 로딩 (외우세요!)
# from tensorflow.keras import datasets
# mnist = datasets.mnist

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [20]:
from tensorflow.keras import datasets #tensorflow.keras 라이브러리에서 datasets 가져오기
mnist = datasets.mnist #mnist 데이터 세트를 mnist 변수에 담기
(X_train, y_train), (X_test, y_test) = mnist.load_data() #mmnist.load_data를 (train set), ( test set) 으로 패킹하기

In [19]:
# 패킹한 data set들의 shape 확인 해보자!
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [28]:
TRAIN_IMAGE_SIZE = X_train.shape[0] #X_train dataset shape의 0번째 인덱스 값(60000)을 변수 TRAIN_IMAGE_SIZE 담아주자

In [29]:
TRAIN_IMAGE_SIZE

60000

In [26]:
X_train.reshape(TRAIN_IMAGE_SIZE, -1) #reshape을 활용한 평탄화 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [31]:
X_train.shape # reshape 된 결과 확인, 

(60000, 784)

# y_train One Hot Encoding 

무계획 coding은 NO!
엔코딩 순서를 먼저 생각한 후 순서에 맞춰서 코딩 구현하기!
- 배열에 대한 이해가 기초! (행/열)에 대한 기초 변환 과정을 알아야 함

1. 1차원 배열을 
2. 열백터 형식으로 변환(reshape 사용)
3. 원핫엔 코딩

In [41]:
y_train #배열 직접 확인 해보기 (1차원 행백터 였음)

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [44]:
y_train1 = y_train.reshape(-1,1) # y_train을 reshape
y_train1 # 배열 찍어서 확인 해보기 (1차원 열백터로 바뀜)

array([[5],
       [0],
       [4],
       ...,
       [5],
       [6],
       [8]], dtype=uint8)

In [45]:
y_train1.shape # shape 확인

(60000, 1)

In [46]:
from sklearn.preprocessing import OneHotEncoder #사이킷런의 one hot encoder 불러오기
y_train_one_hot = OneHotEncoder().fit_transform(y_train1) # y_train1을 OHE로 fit(훈련) 시켜서 변환
y_train_one_hot #sparse row format 압축된 형태로 나옴

<60000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 60000 stored elements in Compressed Sparse Row format>

In [ ]:
y_train_one_hot_arr = y_train_one_hot.toarray()
y_train_one_hot_arr.shape

(60000, 10)

In [ ]:
y_train_one_hot_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
y_train_one_hot_arr[:4]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

# 미니 배치 구현

In [ ]:
#열심히 안봐도 된다고 하심)
import numpy as np

#훈련 데이터에서 무작위 10장 추출
train_size = X_train.shape[0]  # 전체 훈련 데이터 세트 크기
batch_size = 10  # 미니 배치의 사이즈

batch_mask = np.random.choice(train_size, batch_size)

batch_mask 

array([58208, 15414, 10559, 21461, 23731, 38054, 14789, 39458, 50112,
       59958])

784

In [ ]:
X_batch = X_train[batch_mask]
t_batch = y_train_one_hot_arr[batch_mask]

X_batch.shape, t_batch.shape

((10, 784), (10, 10))

In [ ]:
# ver 1 - OHE 되어 있는 형태
def cross_entropy_error_v1(y, t):
    delta = 1e-6
#배치를 사용하지 않은 경우
if y.ndim == 1:
    # batch size를 강제로 1로 지정
    # reshape으로
    t = t.reshape(1, -1)
    y = y.reshape(1, -1)

batch_size = y.shape[0]
return =np.sum(t * np.log(y+delta)) / batch_size 

In [ ]:
# ver 2 - t가 OHE 안되어 있는 형태

def cross_entropy_error_v2(y, t):
    if y.ndim ==1: 
        t = t.reshape(1, t.size) 
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size 
    # - 마이너스화
    # Sum 합
    # np.log로그

    batch_size로 랜덤 추출 

In [ ]:
batch_size

10

In [ ]:
t = t.reshape(1, t.size) 

In [ ]:
#제일 안쪾에 있는 변수 
# 예시를 만들어보기 
batch_size : 5 #y:예측값) 
t :[2, ] #t 타겟값

In [70]:
t = np.array([0, 0, 1,   0,    0,   0,   0,   0,   0,   0])

In [71]:
t

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [72]:
type(t)

numpy.ndarray